### Sindicâncias e inquéritos administrativos na Corregedoria da Guarda Civil Metropolitana

**Autor:** Rodolfo Viana<br>
**Data de início:** 14 de março de 2018<br>
**Data de término:** 14 de março de 2018<br>
**Fonte de dados:** Corregedoria da GCM<br>
**Coleta de dados:** Documento `.pdf` obtido via Lei de Acesso à Informação, em 5 de fevereiro de 2018<br>
**Arquivo de origem (convertido):** `gcm.csv`<br>
**Ferramentas e bibliotecas:** Python 3.6, Jupyter, Pandas

In [1]:
# Importação de ferramentas
import pandas as pd

# Criação de dataset
dataset = pd.read_csv('gcm.csv', sep = ';', encoding = 'utf-8')

In [2]:
# Checagem de dataset
dataset.shape

(550, 6)

In [3]:
# Checagem de tipos
dataset.dtypes

NºProc                  int64
Tipo                   object
DataAutuação           object
Dt Dos Fatos           object
Penalidade Recebida    object
Natureza               object
dtype: object

In [4]:
# Checagem quantitativa
dataset.count()

NºProc                 550
Tipo                   550
DataAutuação           550
Dt Dos Fatos           392
Penalidade Recebida    550
Natureza               549
dtype: int64

In [5]:
# Checagem de registro aleatório
dataset.iloc[43]

NºProc                 201701547950
Tipo                    SINDICÂNCIA
DataAutuação             09/10/2017
Dt Dos Fatos             20/03/2017
Penalidade Recebida    EM ANDAMENTO
Natureza                   agressão
Name: 43, dtype: object

A checagem preliminar mostra que:
1. o dataset agrega 550 registros;
2. um dos 550 registros não contém a natureza (o motivo do processo administrativo);
3. 158 dos 550 registros não contêm a data dos fatos (quando ocorreu o suposto desvio de conduta);
4. os headers contêm espaços e símbolos;
5. as datas não estão no formato `datetime`.

Vamos arrumar tudo isso.

In [6]:
# Transformação das datas no padrão para análise
dataset['DataAutuação'] = pd.to_datetime(dataset['DataAutuação'])
dataset['Dt Dos Fatos'] = pd.to_datetime(dataset['Dt Dos Fatos'])

# Renomeação das colunas
dataset = dataset.rename(columns={
    'NºProc': 'num_processo', 
    'Tipo': 'tipo', 
    'DataAutuação': 'data_autuacao', 
    'Dt Dos Fatos': 'data_fato', 
    'Penalidade Recebida': 'penalidade', 
    'Natureza': 'motivo'
})

# Criação dos campos 'ano_autuacao' e 'ano_fato' para futuras análises temporais
dataset['ano_autuacao'] = dataset['data_autuacao'].dt.year
dataset['ano_fato'] = dataset['data_fato'].dt.year

In [7]:
# Checagem de registro aleatório
dataset.iloc[43]

num_processo            201701547950
tipo                     SINDICÂNCIA
data_autuacao    2017-09-10 00:00:00
data_fato        2017-03-20 00:00:00
penalidade              EM ANDAMENTO
motivo                      agressão
ano_autuacao                    2017
ano_fato                        2017
Name: 43, dtype: object

In [8]:
# Contagem dos tipos de processos
dataset['tipo'].value_counts()

SINDICÂNCIA                 368
INQUÉRITO ADMINISTRATIVO    182
Name: tipo, dtype: int64

In [9]:
# Separação da sindicância
sindicancia = dataset[dataset['tipo'] == 'SINDICÂNCIA']

# Contagem de penalidades
sindicancia['penalidade'].value_counts()

ARQUIVAMENTO                      201
EM ANDAMENTO                       97
APLICAÇÃO DIRETA DE PENALIDADE     34
INQUÉRITO ADMINISTRATIVO           18
ARQUIVAMENTO PELA DUPLICIDADE       5
ACOLHE A DEFESA                     4
ADVERTÊNCIA                         3
REPREENSÃO                          3
PROCESSO SUMÁRIO                    1
EXTINÇÃO POR PRESCRIÇÃO             1
SUSPENSÃO                           1
Name: penalidade, dtype: int64

Notamos que, dentre os 368 processos do tipo sindicância, temos:

- 206 (55,9% do total) arquivados (inclusive por duplicidade);
- 97 (26,3%) em andamento;
- 42 (11,4%) com alguma punição (aplicação direta de penalidade, advertência, repreensão, processo sumário e suspensão);
- 18 (4,9%) que se tornaram inquéritos administrativos;
- 1 (0,2%) extinto por prescrição.

Cabe ressaltar que são processos de 2016 e 2017. De acordo com a [lei 8.989, de 1979](http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/negocios_juridicos/LEI%208989%2079.pdf) (Estatuto dos Funcionários Públicos do Município de São Paulo),

> **Art. 206**: A sindicância deverá estar concluída no prazo de trinta dias, que só poderá ser prorrogado mediante justificação fundamentada.

Assim, vale um recorte mais detalhado nos processos até 30 de novembro de 2017 -- que, em tese, já deveriam ter sido concluídos até janeiro de 2018 -- para ver esse percentual.

In [10]:
# Separação da sindicância do período
sind_ate_novembro = sindicancia[sindicancia['data_autuacao'] <= '2017-11-30 00:00:00']

# Contagem de penalidades
sind_ate_novembro['penalidade'].value_counts()

ARQUIVAMENTO                      201
EM ANDAMENTO                       84
APLICAÇÃO DIRETA DE PENALIDADE     34
INQUÉRITO ADMINISTRATIVO           18
ARQUIVAMENTO PELA DUPLICIDADE       5
ACOLHE A DEFESA                     4
ADVERTÊNCIA                         3
REPREENSÃO                          3
PROCESSO SUMÁRIO                    1
EXTINÇÃO POR PRESCRIÇÃO             1
SUSPENSÃO                           1
Name: penalidade, dtype: int64

O recorte mostra que:

- 201 processos (56,6% do total) foram arquivados -- ou seja, o guarda civil foi inocentado;
- 84 (23,7%) ainda estavam em andamento em janeiro de 2018, contrariando a legislação, se não houver "justificação fundamentada" para a prorrogação;
- 42 (11,8%) tiveram alguma punição; e
- 18 (5,1%) tornaram-se inquéritos administrativos.

Cabe ressaltar que, também segundo o estatuto do servidor municipal,

> **Art. 207**: Instaura-se inquérito administrativo quando a falta disciplinar, por sua natureza, possa determinar a pena de demissão.

Vejamos se na lista de inquéritos há processos já contabilizados na lista de sindicâncias:

In [11]:
# Separação do inquérito
inquerito = dataset[dataset['tipo'] == 'INQUÉRITO ADMINISTRATIVO']

# Procura por número de processo em ambas as listas
registros_replicados = inquerito[inquerito['num_processo'].isin(sindicancia['num_processo'])]

# Resultado
registros_replicados.shape

(0, 8)

Não há processo cujo número esteja nas listas de inquérito e sindicância. Duas hipóteses:

1. uma sindicância, ao se tornar inquérito, recebe outro número de processo; ou
2. nenhum inquérito aberto no período teve, como origem, sindicância no mesmo período.

Qualquer que seja a resposta, não afeta a análise a partir de agora, pois o foco é apenas inquérito, independentemente de sua origem -- sindicância ou não.

In [12]:
# Contagem de penalidades
inquerito['penalidade'].value_counts()

EM ANDAMENTO                           146
SUSPENSÃO                               13
DEMISSÃO                                 5
APLICAÇÃO DIRETA DE PENALIDADE           4
CONVERTE O JULGAMENTO EM DILIGÊNCIA      4
ARQUIVAMENTO                             3
EXTINÇÃO SEM JULGAMENTO DO MÉRITO        3
DECISÃO EM OUTRO PA                      1
CASSAÇÃO DA APOSENTADORIA                1
SOBRESTADO                               1
DEMISSÃO À BEM DO SERVIÇO PÚBLICO        1
Name: penalidade, dtype: int64

Notamos que, dentre os 182 inquéritos, temos:

- 146 (80,2% do total) em andamento;
- 17 (9,3%) suspensões, incluindo aplicação direta de penalidade (que, conforme informado pela GCM, é suspensão);
- 6 (3,3%) demissões, incluindo uma demissão a bem do serviço público;
- 3 (1,6%) arquivados;
- 1 (0,5%) cassação de aposentaria.

Um destaque são os três casos que foram extintos sem julgamento do mérito. De acordo com o estatuto do servidor municipal,

> **Art. 209**: O inquérito administrativo será iniciado no prazo de 5 (cinco) dias, contados do recebimento dos autos pela Comissão Processante e concluído no prazo de 90 (noventa) dias, contados do seu início.

> § 1º. O prazo para a conclusão do inquérito poderá ser prorrogado, a juízo da autoridade que determinou sua instauração, mediante justificação fundamentada.

Com o prazo de 90 dias, seria injusto se esta análise, cujos dados foram solicitados em janeiro de 2018, contabilizasse inquéritos abertos em data posterior a 30 de setembro de 2017.

In [13]:
# Separação da inquérito do período
inq_ate_setembro = inquerito[inquerito['data_autuacao'] <= '2017-09-30 00:00:00']

# Contagem de penalidades
inq_ate_setembro['penalidade'].value_counts()

EM ANDAMENTO                           130
SUSPENSÃO                               13
DEMISSÃO                                 5
CONVERTE O JULGAMENTO EM DILIGÊNCIA      4
APLICAÇÃO DIRETA DE PENALIDADE           3
ARQUIVAMENTO                             3
EXTINÇÃO SEM JULGAMENTO DO MÉRITO        3
CASSAÇÃO DA APOSENTADORIA                1
SOBRESTADO                               1
DECISÃO EM OUTRO PA                      1
DEMISSÃO À BEM DO SERVIÇO PÚBLICO        1
Name: penalidade, dtype: int64

O recorte de tempo resulta em 17 inquéritos a menos, sendo 16 deles no status "em andamento". Ou seja, pouca variação do resultado anterior.

Vamos descartar as sindicâncias e os inquéritos em andamento para ver quais os desvios de conduta mais comuns que já tiveram algum resultado -- e quais foram esses resultados: 

In [14]:
# Concatenação de sindicâncias e inquéritos até os períodos-limite
dataset_corte = pd.concat([sind_ate_novembro, inq_ate_setembro])

# Recorte de todos os processos que não estão em andamento
dataset_corte = dataset_corte[dataset_corte['penalidade'] != 'EM ANDAMENTO']

# Contagem de ocorrências por motivo (para fazer a porcentagem depois)
dataset_corte['chave'] = dataset_corte.groupby(['motivo'])['motivo'].transform('count')

# Agrupamento por motivo e contagem de penalidades e não-penalidades
dataset_corte_grupo = dataset_corte.groupby(['motivo', 'penalidade', 'chave'])\
                                   .agg({'penalidade': 'count'})\
                                   .sort_values('penalidade', ascending = False)

# Transformação em DataFrame        
dataset_corte_grupo = pd.DataFrame(dataset_corte_grupo)\
                        .rename(columns = {'penalidade': 'quantidade'})\
                        .reset_index()

# Resultado
dataset_corte_grupo['motivo'].unique()

array(['agressão', 'ato incompatível com a função',
       'abuso de autoridade / poder', 'acúmulo indevido de cargo',
       'lesão corporal', 'descumprimento de ordem', 'outros',
       'exercício de atividade laboral em LM',
       'crimes contra honra: calúnia, injúria e difamação',
       'faltar com a verdade', 'faltas consecutivas', 'assédio moral',
       'ameaça', 'prevaricação', 'apreensão de mercadorias/objetos',
       'falta de urbanidade', 'fuga de preso', 'homicídio',
       'violência doméstica/Lei Maria da Penha',
       'denegrir imagem da corporação', 'captura de procurado',
       'acidente de trânsito/colisão/choque', 'disparo de arma de fogo',
       'desacato', 'danos à vtr', 'danos ao erário público / patrimônio',
       'porte ilegal de arma', 'pertubação da paz alheia',
       'faltas interpoladas',
       'lesão corporal na direção de veículo automotor',
       'homicídio culposo', 'maus tratos', 'homicídio doloso',
       'violação de domicílio', 'corrupção/

Vamos ver quais desvios de conduta levaram guardas civis à demissão:

In [15]:
# Recorte da punição
demissao = dataset_corte_grupo[(dataset_corte_grupo['penalidade'] == 'DEMISSÃO') |\
                               (dataset_corte_grupo['penalidade'] == 'DEMISSÃO À BEM DO SERVIÇO PÚBLICO')]

# Resultado
demissao

,motivo,penalidade,chave,quantidade
11,faltas consecutivas,DEMISSÃO,8.0,4
53,homicídio culposo,DEMISSÃO À BEM DO SERVIÇO PÚBLICO,2.0,1
81,exercício de atividade laboral em LM,DEMISSÃO,14.0,1


E homicídios?

In [16]:
# Recorte do motivo
homicidio = dataset_corte_grupo[(dataset_corte_grupo['motivo'] == 'homicídio doloso') |\
                                (dataset_corte_grupo['motivo'] == 'homicídio culposo') |\
                                (dataset_corte_grupo['motivo'] == 'homicídio')]

# Resultado
homicidio

,motivo,penalidade,chave,quantidade
22,homicídio,ARQUIVAMENTO,4.0,3
50,homicídio,CONVERTE O JULGAMENTO EM DILIGÊNCIA,4.0,1
51,homicídio culposo,ARQUIVAMENTO,2.0,1
53,homicídio culposo,DEMISSÃO À BEM DO SERVIÇO PÚBLICO,2.0,1
54,homicídio doloso,CONVERTE O JULGAMENTO EM DILIGÊNCIA,1.0,1


In [17]:
# Criação de uma coluna chamada 'punido' e preenchimento 
# de cada linha da coluna com 'Sim', 'Não' e 'Incerto' 
# (quando o resultado é a continuação do processo), de 
# acordo com a penalidade
punido = []

for row in dataset_corte['penalidade']:
    if row == 'EM ANDAMENTO':
        punido.append('Incerto')        
    elif row == 'DECISÃO EM OUTRO PA':
        punido.append('Incerto')
    elif row == 'INQUÉRITO ADMINISTRATIVO':
        punido.append('Incerto')
    elif row == 'ARQUIVAMENTO':
        punido.append('Não')
    elif row == 'EXTINÇÃO SEM JULGAMENTO DO MÉRITO':
        punido.append('Não')
    elif row == 'SOBRESTADO':
        punido.append('Não') 
    elif row == 'ARQUIVAMENTO PELA DUPLICIDADE':
        punido.append('Não')
    elif row == 'EXTINÇÃO POR PRESCRIÇÃO':
        punido.append('Não') 
    elif row == 'ACOLHE A DEFESA':
        punido.append('Não') 
    else:
        punido.append('Sim')

# Cópia da coluna no DataFrame original
dataset_corte['punido'] = punido

Tomando todos os tipos de desvios de conduta, quantos tiveram punições?

In [18]:
dataset_corte['punido'].value_counts()

Não        218
Sim         69
Incerto     19
Name: punido, dtype: int64

Vamos ver o percentual de punição para cada motivo:

In [19]:
# Agrupamento dos dados por motivo e punição
final = dataset_corte.groupby(['motivo', 'chave'])['punido'].value_counts()

# Transformação em DataFrame
final = pd.DataFrame(final).rename(columns = {'punido': 'quantidade'}).reset_index()

# Definição da porcentagem
final['porcentagem'] = (final['quantidade'] / final['chave']) * 100

# Ajuste da porcentagem para duas casas decimais
final['porcentagem'] = final['porcentagem'].map('{0:.2f}'.format)

# Dispensa da coluna 'chave'
final = final.drop(columns = 'chave', axis = 1)

# Ordenação alfabética
final = final.sort_values('motivo', ascending = True)

# Resultado
final

,motivo,punido,quantidade,porcentagem
0,abuso de autoridade / poder,Não,17,100.00
1,acidente de trânsito/colisão/choque,Não,2,100.00
2,acúmulo indevido de cargo,Sim,12,70.59
3,acúmulo indevido de cargo,Não,5,29.41
4,adulteração de sinal de veículo automotor,Sim,1,100.00
5,agressão,Não,73,98.65
6,agressão,Sim,1,1.35
7,ameaça,Não,5,100.00
8,apreensão de mercadorias/objetos,Não,3,100.00
9,assédio moral,Não,4,57.14


In [20]:
# Gravação em arquivo
final.to_csv('gcm_resultados.csv', sep = ';', encoding = 'utf-8', index = False)